<a href="https://colab.research.google.com/github/joeycarlin3/ai-poc-team-h2p/blob/main/app_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types
from google.colab import userdata


def generate():
    client = genai.Client(
        api_key = userdata.get("GOOGLE_API_KEY"),
    )

    model = "gemini-2.5-pro"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""INSERT_INPUT_HERE"""),
            ],
        ),
    ]
    tools = [
        types.Tool(googleSearch=types.GoogleSearch(
        )),
    ]
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        tools=tools,
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate()

Hello! It seems you're ready to ask a question or give me a command. Please go ahead and type your request in the space where you see "INSERT_INPUT_HERE".

I can help you with a variety of tasks, such as:

*   **Answering questions:** "What is the capital of France?"
*   **Summarizing text:** "Summarize the plot of the movie 'Inception'."
*   **Writing creative content:** "Write a short poem about a rainy day."
*   **Translating languages:** "Translate 'hello' into Spanish."
*   **And much more!**

I'm here to assist you. What would you like to know or do?

In [3]:
pip install streamlit google-generativeai streamlit-mic-recorder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 31.7 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import google.generativeai as genai
from streamlit_mic_recorder import mic_recorder

# --- Configuration ---
st.set_page_config(page_title="AI Interview Coach", page_icon="🎙️")

# --- Gemini API Configuration ---
try:
    # Get the API key from Streamlit secrets
    GEMINI_API_KEY = st.secrets["GOOGLE_API_KEY"]
    genai.configure(api_key=GEMINI_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
except KeyError:
    st.error("Gemini API key not found. Please add it to your Streamlit secrets.")
    st.stop()
except Exception as e:
    st.error(f"An error occurred during Gemini API configuration: {e}")
    st.stop()


# --- Functions ---
def generate_behavioral_question(career_goal):
    """
    Generates a behavioral interview question based on the user's career goal.
    """
    prompt = f"Generate a behavioral interview question for a candidate aspiring to be a {career_goal}."
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        st.error(f"Error generating question: {e}")
        return "Tell me about a time you had to handle a difficult stakeholder."

def get_feedback(question, answer):
    """
    Uses the Gemini API to get feedback on the user's answer.
    """
    prompt = f"""
    Provide feedback on the following interview response.
    The question was: "{question}"
    The user's answer is: "{answer}"

    Analyze the answer based on the STAR method (Situation, Task, Action, Result) and provide constructive feedback.
    """
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        st.error(f"Error generating feedback: {e}")
        return "Could not generate feedback at this time."

# --- Streamlit App ---
st.title("AI-Powered Interview Feedback Tool")

st.write(
    "Practice your behavioral interview skills. Enter your career goal, get a relevant question, "
    "record your answer, and receive AI-powered feedback."
)

# --- 1. Get Career Goal ---
career_goal = st.text_input("Enter your career goal (e.g., Software Engineer, Product Manager):")

if career_goal:
    # --- 2. Generate and Display Question ---
    if 'question' not in st.session_state or st.button("Get a New Question"):
        st.session_state.question = generate_behavioral_question(career_goal)

    st.subheader("Your behavioral question is:")
    st.write(st.session_state.question)

    # --- 3. Record User's Response ---
    st.write("Record your answer:")
    audio = mic_recorder(
        start_prompt="Start Recording",
        stop_prompt="Stop Recording",
        key='recorder'
    )

    if audio:
        st.audio(audio['bytes'])
        # For simplicity, we will use a placeholder for transcription.
        # In a real-world application, you would integrate a speech-to-text API.
        # Here, we will ask the user to type their answer for feedback.
        st.subheader("Transcribe your answer for feedback:")
        user_answer = st.text_area("Your transcribed answer:", height=200)

        # --- 4. Get and Display Feedback ---
        if st.button("Get Feedback"):
            if user_answer:
                with st.spinner("Generating feedback..."):
                    feedback = get_feedback(st.session_state.question, user_answer)
                    st.subheader("Feedback on Your Answer:")
                    st.write(feedback)
            else:
                st.warning("Please transcribe your answer to get feedback.")
